In [97]:
import pandas as pd
import plotly.express as px
import json

In [31]:
data = pd.read_csv('data/time_series_covid19_confirmed_global_iso3_regions.csv')

In [32]:
data.drop(index=0, inplace=True)

In [33]:
data.columns[:4]

Index(['Province/State', 'Country/Region', 'Lat', 'Long'], dtype='object')

In [34]:
data.columns[-7:]

Index(['ISO 3166-1 Alpha 3-Codes', 'Region Code', 'Region Name',
       'Sub-region Code', 'Sub-region Name', 'Intermediate Region Code',
       'Intermediate Region Name'],
      dtype='object')

In [45]:
regions = data.drop(columns=data.columns[:4]).drop(columns=['ISO 3166-1 Alpha 3-Codes', 'Region Code','Sub-region Code', 'Sub-region Name', 'Intermediate Region Code','Intermediate Region Name']).groupby('Region Name').sum().T

In [48]:
regions.columns.name=None

In [52]:
regions.index = pd.to_datetime(regions.index, format='%m/%d/%y') 

In [53]:
regions

,Africa,Americas,Asia,Europe,Oceania
2020-01-22,0.0,1.0,556.0,0.0,0.0
2020-01-23,0.0,3.0,654.0,0.0,0.0
2020-01-24,0.0,5.0,937.0,2.0,0.0
2020-01-25,0.0,5.0,1429.0,3.0,0.0
2020-01-26,0.0,8.0,2105.0,3.0,4.0
...,...,...,...,...,...
2023-03-05,13055369.0,191044835.0,211313286.0,246585327.0,13999990.0
2023-03-06,13056372.0,191053527.0,211331424.0,246615532.0,13999992.0
2023-03-07,13056975.0,191097450.0,211359172.0,246673695.0,13999992.0
2023-03-08,13058586.0,191179187.0,211394742.0,246734062.0,14000153.0


In [59]:
px.line(regions.diff().applymap(lambda x : max(0,x)), y=regions.columns, height=800)

/var/folders/3v/836xbswd3z1f339m_2k6w7b80000gn/T/ipykernel_10873/2347106406.py:1: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [ ]:
countries = data.set_index(['ISO 3166-1 Alpha 3-Codes',"Country/Region"]).drop(columns=['Province/State','Lat', 'Long']).drop(columns=['Region Name', 'Region Code','Sub-region Code', 'Sub-region Name', 'Intermediate Region Code','Intermediate Region Name']).T

In [ ]:
countries.index = pd.to_datetime(countries.index, format='%m/%d/%y') 

In [ ]:
countries

In [ ]:
countries = countries.resample('Q').sum().reset_index().melt(id_vars=['index'])

In [ ]:
countries

In [ ]:
countries.columns = ['Date','ISO_A3','Country/Region','Cases']

In [ ]:
countries

In [ ]:
GeoJson = json.load(open("data/countries.geojson", "r"))

In [ ]:
fig = px.choropleth_mapbox(
    countries,
    geojson=GeoJson,
    locations='ISO_A3',
    color='Cases',
    featureidkey="properties.ISO_A3",
    zoom=1,
    mapbox_style="open-street-map",
    width=1600,
    height=900,
    opacity=0.5,
    animation_frame='Date',
    color_continuous_scale="amp",
    range_color=(0,450000000)
    
)

In [ ]:
fig.write_html('Maps_20_23.html')